In [ ]:
import pandas as pd

# 读取 parquet 文件
df = pd.read_csv("TruthfulQA.csv")

# 各列保存为列表
types = df["Type"].tolist()
categories = df["Category"].tolist()
questions = df["Question"].tolist()
best_answers = df["Best Answer"].tolist()
correct_answers = df["Correct Answers"].tolist()
incorrect_answers = df["Incorrect Answers"].tolist()
sources = df["Source"].tolist()
correct_answers = [c.split("; ") for c in correct_answers]
incorrect_answers = [ic.split("; ") for ic in incorrect_answers]
# 简单检查
print(best_answers[0])
print(correct_answers[0]) 
print(incorrect_answers[0])
print(len(questions))

In [ ]:
import easysteer.hidden_states as hs
from vllm import LLM
import os
os.environ["VLLM_USE_V1"] = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
llm = LLM(
    model="/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/",
    task="reward", 
    tensor_parallel_size=1
)

In [ ]:
positive = []
for i in range(408):
    for j in range(len(correct_answers[i])):
        test_str = f"""<|im_start|>user
{questions[i]}
<|im_end|>\n<|im_start|>assistant
{correct_answers[i][j]}.
"""
        positive.append(test_str)


negative = []
for i in range(408):
    for j in range(len(incorrect_answers[i])):
        test_str = f"""<|im_start|>user
{questions[i]}
<|im_end|>\n<|im_start|>assistant
{incorrect_answers[i][j]}.
"""
        negative.append(test_str)

In [ ]:
all_hidden_states, outputs = hs.get_all_hidden_states(llm, positive+negative)

In [ ]:
from easysteer.steer import extract_diffmean_control_vector, StatisticalControlVector
control_vector = extract_diffmean_control_vector(
    all_hidden_states=all_hidden_states, 
    positive_indices=list(range(len(positive))),  
    negative_indices=list(range(len(positive),len(positive+negative))),  
    model_type="qwen",
    token_pos=-1,
    normalize=True
)
control_vector.export_gguf("real1-caa.gguf")

from easysteer.steer import extract_pca_control_vector, StatisticalControlVector
control_vector = extract_pca_control_vector(
    all_hidden_states=all_hidden_states, 
    positive_indices=list(range(len(positive))),  
    negative_indices=list(range(len(positive),len(positive+negative))),  
    model_type="qwen2.5",
    method="center",
    token_pos=-1, 
    normalize=True
)
control_vector.export_gguf("real1-pca.gguf")

from easysteer.steer import extract_linear_probe_control_vector, StatisticalControlVector
control_vector = extract_linear_probe_control_vector(
    all_hidden_states=all_hidden_states, 
    positive_indices=list(range(len(positive))),  
    negative_indices=list(range(len(positive),len(positive+negative))),  
    model_type="qwen2.5",
    token_pos=-1, 
    normalize=True
)
control_vector.export_gguf("real1-probe.gguf")

In [ ]:
positive = []
for i in range(408,817):
    for j in range(len(correct_answers[i])):
        test_str = f"""<|im_start|>user
{questions[i]}
<|im_end|>\n<|im_start|>assistant
{correct_answers[i][j]}
"""
        positive.append(test_str)


negative = []
for i in range(408,817):
    for j in range(len(incorrect_answers[i])):
        test_str = f"""<|im_start|>user
{questions[i]}
<|im_end|>\n<|im_start|>assistant
{incorrect_answers[i][j]}
"""
        negative.append(test_str)

In [ ]:
all_hidden_states, outputs = hs.get_all_hidden_states(llm, positive+negative)

In [ ]:
from easysteer.steer import extract_diffmean_control_vector, StatisticalControlVector
control_vector = extract_diffmean_control_vector(
    all_hidden_states=all_hidden_states, 
    positive_indices=list(range(len(positive))),  
    negative_indices=list(range(len(positive),len(positive+negative))),  
    model_type="qwen",
    token_pos=-1,
    normalize=True
)
control_vector.export_gguf("real2-caa.gguf")

from easysteer.steer import extract_pca_control_vector, StatisticalControlVector
control_vector = extract_pca_control_vector(
    all_hidden_states=all_hidden_states, 
    positive_indices=list(range(len(positive))),  
    negative_indices=list(range(len(positive),len(positive+negative))),  
    model_type="qwen2.5",
    method="center",
    token_pos=-1, 
    normalize=True
)
control_vector.export_gguf("real2-pca.gguf")

from easysteer.steer import extract_linear_probe_control_vector, StatisticalControlVector
control_vector = extract_linear_probe_control_vector(
    all_hidden_states=all_hidden_states, 
    positive_indices=list(range(len(positive))),  
    negative_indices=list(range(len(positive),len(positive+negative))),  
    model_type="qwen2.5",
    token_pos=-1, 
    normalize=True
)
control_vector.export_gguf("real2-probe.gguf")